## Imports

In [1]:
import pandas as pd
import pyodbc 
import import_ipynb
import functions as f

importing Jupyter notebook from functions.ipynb


In [2]:
# read datasets

df_cases = pd.read_parquet('./Processed/WHO-COVID19-CASES.parquet')

df_vaccines = pd.read_parquet('./Processed/WHO-COVID19-VACCINES.parquet')

df_countries = pd.read_parquet('./Processed/COUNTRIES_DETAILED.parquet')

In [3]:
# Find the latest date in df_cases and df_vaccines

max_cases_df_date = df_cases['DateReported'].max().date()

max_vaccines_df_date = df_vaccines['Date'].max().date()

In [4]:
# SQL Server connection variables

server = 'lirkovsrv.database.windows.net'
database = 'lirkovdb'
username = 'lirkov'
password = '{LLirkoff31}'   
driver= '{ODBC Driver 17 for SQL Server}'

In [6]:
# upsert to covid.who_cases SQL Server table

# connect to SQL Server
conn =  pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = conn.cursor()

# check the latest date in the database
cursor.execute('SELECT MAX(DateReported) FROM covid.who_cases')
max_db_date = cursor.fetchone()

# compare dates and insert missing records if any
if max_db_date[0] < max_cases_df_date:
    df_cases_upsert = df_cases[df_cases['DateReported'].dt.date > max_db_date[0]]
    rows = 0
    for index, row in df_cases_upsert.iterrows():
        cursor.execute("INSERT INTO covid.who_cases (DateReported, Country, IsoCode, WhoRegion, NewCases, CumulativeCases, NewDeaths, CumulativeDeaths) VALUES(?,?,?,?,?,?,?,?)", 
        row.DateReported, row.Country, row.IsoCode, row.WhoRegion, row.NewCases, row.CumulativeCases, row.NewDeaths, row.CumulativeDeaths)
        rows += 1
    conn.commit()
    print(f'{rows} rows have been inserted into covid.who_cases')
else:
    print('covid.who_cases is up to date')

cursor.close()

872 rows have been inserted


In [10]:
# upsert to covid.who_vaccines SQL Server table

# connect to SQL Server
conn =  pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = conn.cursor()

# check the latest date in the database
cursor.execute('SELECT MAX(Date) FROM covid.who_vaccines')
max_db_date = cursor.fetchone()

try:
    # compare dates and insert missing records if any
    if max_db_date[0] < max_vaccines_df_date:
        df_vaccines_upsert = df_vaccines[df_vaccines['Date'].dt.date > max_db_date[0]]
        rows = 0
        for index, row in df_vaccines_upsert.iterrows():
            cursor.execute("INSERT INTO covid.who_vaccines (Location, IsoCode, Date, TotalVaccinations, PeopleVaccinated, PeopleFullyVaccinated, TotalBoosters, DailyVaccinationsRaw, DailyVaccinations, TotalVaccinationsPerHundred, PeopleVaccinatedPerHundred, PeopleFullyVaccinatedPerHundred, TotalBoostersPerHundred, DailyVaccinationsPerMillion, DailyPeopleVaccinated, DailyPeopleVaccinatedPerHundred) VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)",
            row.Location, row.IsoCode, row.Date, row.TotalVaccinations, row.PeopleVaccinated, row.PeopleFullyVaccinated, row.TotalBoosters, row.DailyVaccinationsRaw, row.DailyVaccinations, row.TotalVaccinationsPerHundred, row.PeopleVaccinatedPerHundred, row.PeopleFullyVaccinatedPerHundred, row.TotalBoostersPerHundred, row.DailyVaccinationsPerMillion, row.DailyPeopleVaccinated, row.DailyPeopleVaccinatedPerHundred)
            rows += 1
        conn.commit()
        print(f'{rows} rows have been inserted into covid.who_vaccines')
    else:
        print('covid.who_vaccines is up to date')
except TypeError: # if inserting is done for the first time or the database table is empty for some reason insert the whole dataframe
    rows = 0
    for index, row in df_vaccines.iterrows():
        cursor.execute("INSERT INTO covid.who_vaccines (Location, IsoCode, Date, TotalVaccinations, PeopleVaccinated, PeopleFullyVaccinated, TotalBoosters, DailyVaccinationsRaw, DailyVaccinations, TotalVaccinationsPerHundred, PeopleVaccinatedPerHundred, PeopleFullyVaccinatedPerHundred, TotalBoostersPerHundred, DailyVaccinationsPerMillion, DailyPeopleVaccinated, DailyPeopleVaccinatedPerHundred) VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)",
        row.Location, row.IsoCode, row.Date, row.TotalVaccinations, row.PeopleVaccinated, row.PeopleFullyVaccinated, row.TotalBoosters, row.DailyVaccinationsRaw, row.DailyVaccinations, row.TotalVaccinationsPerHundred, row.PeopleVaccinatedPerHundred, row.PeopleFullyVaccinatedPerHundred, row.TotalBoostersPerHundred, row.DailyVaccinationsPerMillion, row.DailyPeopleVaccinated, row.DailyPeopleVaccinatedPerHundred)
        rows += 1
        if rows == 10000:
            conn.commit()
            print(f'{rows} commited')

cursor.close()


202 rows have been inserted


In [28]:
# # insert countries

# # connect to SQL Server
# conn =  pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
# cursor = conn.cursor()

# # check if there are any records
# cursor.execute('SELECT COUNT(DISTINCT(IsoCode)) FROM covid.countries')
# countries_records = cursor.fetchone()

# # countries data should be inserted ones. If it is missing insert it.
# if int(countries_records) == 0:
#     cursor.execute("INSERT INTO covid.countries (Continent, Location, IsoCode, Population, PopulationDensity, MedianAge, GdpPerCapita, HumanDevelopmentIndex, LifeExpectancy, Aged65Older, Aged70Older, Alpha2) VALUES(?,?,?,?,?,?,?,?,?,?,?,?)",
#         row.Continent, row.Location, row.IsoCode, row.Population, row.PopulationDensity, row.MedianAge, row.GdpPerCapita, row.HumanDevelopmentIndex, row.LifeExpectancy, row.Aged65Older, row.Aged70Older, row.Alpha2)

# conn.commit()
# cursor.close()